**AI-based recommendation system**

Goals:

Steps:

---



#**1. Import libraries**

In [70]:
!pip install openai==0.28

In [71]:
import openai
import requests
import pandas as pd
import pprint

In [77]:
# load data
openai.api_key = ''

# **2. Create collectmovies function**

*From the previous work, we can put everything into function to make it easy to call as the following.*

---



In [73]:
# create a function that collect movies
def collectmovies(params):
    # construct URL
    url = "https://api.themoviedb.org/3/discover/movie?" + "&".join([f"{key}={value}" for key, value in params.items()])
    headers = {
        "accept": "application/json",
        "Authorization": "Bearer "
        }
    response = requests.get(url, headers=headers)

    # check if the request was successful (it will show status code 200)
    if response.status_code == 200:
        # parse the json response
        data = response.json()

        # extract the list of movies from the response
        movies = data.get('results', [])

        # create a dataframe
        df = pd.DataFrame(movies)
    else:
        print('Error fetching data. Status code:', response.status_code)

    # if everything works, it will proceed to the next step

    # map the genre to its ID
    df.drop(columns=["backdrop_path","id","poster_path","title","video","adult","original_language"], inplace=True)
    urls = "https://api.themoviedb.org/3/genre/movie/list?language=en"
    response = requests.get(urls, headers=headers)
    genres_data = response.json()
    genre_map = {genre["id"]: genre["name"] for genre in genres_data.get("genres", [])}
    df["genre_names"] = df["genre_ids"].apply(lambda ids: [genre_map[id] for id in ids])
    df.drop(columns=["genre_ids"], inplace=True)
    return df

# **3. Create movie_list function**

In [74]:
def movie_list(df, genre):
    filtered_list = df[df['genre_names'].apply(lambda x: genre in x)]

    if not filtered_list.empty:
        sorted_df = filtered_list.sort_values(by='vote_average', ascending=False)
        recommendations = [f"{row['original_title']} - {row['vote_average']} - {row['release_date']}" for index, row in sorted_df.iterrows()]
        return recommendations
    else:
        print(f"No movies of genre {genre} found in the list, but the AI can recommend something else.")
        sorted_df = df.sort_values(by='vote_average', ascending=False)
        return sorted_df[['original_title', 'vote_average', 'release_date']]

# **4. Create userinfo function**

In [75]:
def userinfo():
    year = 2020
    region = 'fr'
    without_genres = 'Drama'
    language = 'fr'

    name = input('Your name: ')
    age = float(input('Your age: '))
    movietype = input('Your movie type: ')
    typeperson = input('Your personality: ')

    params = {
        "region": region,
        "without_genres": without_genres,
        "language": language,
        "year": year,
        "page": 10,
        "sort_by": "popularity.desc",
        "name": name,
        "age": age,
        "movietype": movietype,
        "typeperson": typeperson
    }

    return params

# **5. Call functions**

In [82]:
!pip install tiktoken
!pip install cohere
!pip install --upgrade openai

In [84]:
# call functions
params = userinfo()
df = collectmovies(params)

print(f"-----------------------AI recommendation-----------------------")
print(f"-----------------------AI recommendation-----------------------")
print(f"-----------------------AI recommendation-----------------------")
print(f"-----------------------AI recommendation-----------------------")
print(f"-----------------------AI recommendation-----------------------")

movie_recommendations=movie_list(df, params["movietype"])

prompt = f"Recommend a good  movie for {params['name']}, his age is {params['age']}, he is {params['typeperson']}, from this list: {movie_recommendations}"
response = openai.Completion.create(model="text-davinci-003", prompt=prompt, max_tokens=30)
ai_pick = response.choices[0].text.strip()

print(f"AI recommendation: {ai_pick}")

Your name: Tam
Your age: 23
Your movie type: Adventure
Your personality: easy going
-----------------------AI recommendation-----------------------
-----------------------AI recommendation-----------------------
-----------------------AI recommendation-----------------------
-----------------------AI recommendation-----------------------
-----------------------AI recommendation-----------------------


APIRemovedInV1: ignored